In [1]:
#read data from csv
import sys
sys.path.append('..')
import logging
logging.basicConfig(level=logging.WARNING)

from src.llm_alex import Llama
from langchain_core import pydantic_v1
from langchain_core.runnables.base import RunnableParallel, RunnableLambda
from langchain.output_parsers.retry import RetryOutputParser
from langchain_core.output_parsers import JsonOutputParser
from typing import List, Optional, Any, Union
from langchain.schema import BaseOutputParser

from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator

from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate



import pandas as pd
import random 
from time import time 
from tqdm.notebook import tqdm
import numpy as np
import json
import os

from sklearn.metrics import classification_report

train_data = pd.read_csv("data/train_data.csv")
test_data = pd.read_csv("data/test_data.csv")

llm = Llama()
llm(query="test")



cuda


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


'It seems like you just typed a random word "test". Is there something specific you\'d like to test or discuss? I\'m here to help with any questions or topics you\'d like to explore!'

## Few-Shot Prompting

In [2]:
#------------ Helper Methods ------------
def get_prompt_length(llm, str_input):
    messages = [
            {"role": "system", "content": llm._system_msg},
            {"role": "user", "content": str_input},
        ]
    prompt = llm.pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )
    input_tokens_length = len(llm.pipeline.tokenizer.encode(prompt))
    prompt_length = len(prompt)
    return input_tokens_length,prompt_length
def process_row(row):
    start = time() 
    try:
        row['prediction'] = main_chain.invoke({"text": row['text']})
    except ValueError as e:
        #print("Value Error encountered")
        row['prediction'] = np.nan
    end = time()
    row['inference_time'] = end-start
    input_tokens_length, prompt_length = get_prompt_length(llm, few_shot_prompt.format(text=row['text']))
    row['input_tokens_length'] = input_tokens_length
    row['prompt_length'] = prompt_length
    row['prompt'] = few_shot_prompt.format(text=row['text'])
    return row
def get_examples(df, num_shots):
    examples = []
    for label in df.label.unique():
        for i, row in df[df['label'] == label].sample(num_shots, random_state=random_seed).iterrows():
            examples.append({
                "input": row['text'],
                "output": str(label)
            })
            df = df.drop(i)
    return examples

In [4]:
#craft examples
tqdm.pandas()

random.seed(42)
random_seed = 42
n_classes_list = [40]#,30,50,76]
num_shots_list = [1]

for n_classes in tqdm(n_classes_list, desc="Processing n_classes"):
    for num_shots in tqdm(num_shots_list, desc="Processing num_shots", leave=False):
        #sampled_classes = train_data['label'].sample(n_classes, random_state=random_seed, replace=False).values
        sampled_classes = random.sample(test_data['label'].unique().tolist(),n_classes)
        train_data_sub = train_data[train_data['label'].isin(sampled_classes)]
        test_data_sub = test_data[test_data['label'].isin(sampled_classes)]

        #print("*"*50)
        #print(f"Number of sampled classes: {len(sampled_classes)}")
        #print(f"Number of samples in test_data_sub: {len(test_data_sub)}")


        #Achtung: muss hier drin stehen, da sampled_classes verwendet wird. Geht nicht sauberer
        class NumberParser(BaseOutputParser):
            def parse(self, text: str) -> Any:
                try:
                    # Extracting the first number from the text
                    number = int(text.strip())
                    if number not in sampled_classes:
                        raise ValueError(f"Number {number} is not in the list of valid classes.")
                    return number
                except ValueError as e:
                    raise ValueError(f"Failed to parse number: {e}")
        

        examples = get_examples(test_data_sub, num_shots)
        example_prompt = PromptTemplate(
            input_variables=["input", "output"], template="input: {input}\noutput: {output}"
        )
        few_shot_prompt = FewShotPromptTemplate(
            prefix="Your task is to classify a given text by assigning a label from the examples to it, Please ONLY respond with the label as a number. ONLY use labels you see in the examples, don't make up own labels.\nHere are some examples:",
            examples=examples,
            example_prompt=example_prompt,
            suffix="input: {text}.\noutput:",
            input_variables=["text"],
        )
        completion_chain = few_shot_prompt | llm
        
        custom_parser = NumberParser()
        retry_parser = RetryOutputParser(parser=custom_parser, max_retries=3)
        
        main_chain = RunnableParallel(
            completion=completion_chain, prompt_value=few_shot_prompt
        ) | RunnableLambda(lambda x: retry_parser.parse_with_prompt(**x))
        
        
        test_data_sub = test_data_sub.progress_apply(process_row, axis=1)

        #-------- Evaluate and log results -----------
        file_name = f"data/few_shot/test/classes_{n_classes}_shots_{num_shots}"
        if not os.path.exists(file_name):
            os.makedirs(file_name)
        test_data_sub.to_csv(os.path.join(file_name,"predictions.csv"),index=False)
        nan_count = test_data_sub['prediction'].isna().sum()
        test_data_sub['prediction'] = test_data_sub['prediction'].fillna(-1)
        #print(f"Number of NaN values in 'prediction': {nan_count}")
        report = classification_report(test_data_sub['label'], test_data_sub['prediction'], output_dict=True, zero_division=np.nan)
        with open(os.path.join(file_name,"results.json"), "w") as f:
            json.dump(report, f)

Processing n_classes:   0%|          | 0/1 [00:00<?, ?it/s]

Processing num_shots:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1600 [00:00<?, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


## Evaluation 

In [5]:
dir_name = "data/few_shot/test/classes_40_shots_1"
predictions = pd.read_csv(os.path.join(dir_name,"predictions.csv"), index_col=False)
with open(os.path.join(dir_name,"results.json"), "r") as f:
        report = json.load(f)

In [6]:
predictions

,text,label,prediction,inference_time,input_tokens_length,prompt_length,prompt
0,How do I locate my card?,11,41.0,2.139808,905,3540,Your task is to classify a given text by assig...
1,"I still have not received my new card, I order...",11,41.0,1.447461,913,3581,Your task is to classify a given text by assig...
2,I ordered a card but it has not arrived. Help ...,11,41.0,1.450712,911,3569,Your task is to classify a given text by assig...
3,Is there a way to know when my card will arrive?,11,11.0,1.453190,910,3564,Your task is to classify a given text by assig...
4,My card has not arrived yet.,11,11.0,1.455353,904,3544,Your task is to classify a given text by assig...
...,...,...,...,...,...,...,...
1595,"If i'm not in the UK, can I still get a card?",24,24.0,1.496940,913,3561,Your task is to classify a given text by assig...
1596,How many countries do you support?,24,NaN,1.496049,905,3550,Your task is to classify a given text by assig...
1597,What countries do you do business in?,24,NaN,1.495343,906,3553,Your task is to classify a given text by assig...
1598,What are the countries you operate in.,24,NaN,1.497036,905,3554,Your task is to classify a given text by assig...


In [7]:
len(predictions["label"].unique())

40

In [8]:
report

{'-1.0': {'precision': 0.0, 'recall': nan, 'f1-score': 0.0, 'support': 0.0},
 '0.0': {'precision': 0.6,
  'recall': 0.075,
  'f1-score': 0.13333333333333333,
  'support': 40.0},
 '1.0': {'precision': 1.0,
  'recall': 0.8,
  'f1-score': 0.8888888888888888,
  'support': 40.0},
 '2.0': {'precision': 1.0,
  'recall': 0.025,
  'f1-score': 0.04878048780487805,
  'support': 40.0},
 '5.0': {'precision': 1.0,
  'recall': 0.025,
  'f1-score': 0.04878048780487805,
  'support': 40.0},
 '6.0': {'precision': 1.0,
  'recall': 0.05,
  'f1-score': 0.09523809523809523,
  'support': 40.0},
 '7.0': {'precision': 1.0,
  'recall': 0.025,
  'f1-score': 0.04878048780487805,
  'support': 40.0},
 '8.0': {'precision': 1.0,
  'recall': 0.1,
  'f1-score': 0.18181818181818182,
  'support': 40.0},
 '11.0': {'precision': 1.0,
  'recall': 0.375,
  'f1-score': 0.5454545454545454,
  'support': 40.0},
 '13.0': {'precision': 0.6521739130434783,
  'recall': 0.375,
  'f1-score': 0.47619047619047616,
  'support': 40.0},
 '14

## Similarity Enhanced Few-Shot Prompting

Idee: wähle few shot examples basierend auf der similarity zur query aus -> reduziere damit kontextlänge.
- FAISS vector storage
- HuggingFace Embeddings

In [51]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS


model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)
from langchain_core.example_selectors import SemanticSimilarityExampleSelector

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    hf,
    FAISS,
    k=3,
)

/pfs/data5/home/kit/anthropomatik/yy5819/llm_classification/venv/lib64/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [62]:
random_seed = 42
n_classes_list = [5,10]
num_shots_list = [1]

for n_classes in tqdm(n_classes_list, desc="Processing n_classes"):
    for num_shots in tqdm(num_shots_list, desc="Processing num_shots", leave=False):
        #sampled_classes = train_data['label'].sample(n_classes, random_state=random_seed).values
        sampled_classes = random.sample(test_data['label'].unique().tolist(),n_classes)
        train_data_sub = train_data[train_data['label'].isin(sampled_classes)]
        test_data_sub = test_data[test_data['label'].isin(sampled_classes)]

        #print("*"*50)
        #print(f"Number of sampled classes: {len(sampled_classes)}")
        #print(f"Number of samples in test_data_sub: {len(test_data_sub)}")


        #Achtung: muss hier drin stehen, da sampled_classes verwendet wird. Geht nicht sauberer
        class NumberParser(BaseOutputParser):
            def parse(self, text: str) -> Any:
                try:
                    # Extracting the first number from the text
                    number = int(text.strip())
                    if number not in sampled_classes:
                        raise ValueError(f"Number {number} is not in the list of valid classes.")
                    return number
                except ValueError as e:
                    raise ValueError(f"Failed to parse number: {e}")
        

        examples = get_examples(test_data_sub, num_shots)
        example_prompt = PromptTemplate(
            input_variables=["input", "output"], template="input: {input}\noutput: {output}"
        )
        few_shot_prompt = FewShotPromptTemplate(
            prefix="Your task is to classify a given text by assigning a label from the examples to it, Please ONLY respond with the label as a number. ONLY use labels you see in the examples, don't make up own labels.\nHere are some examples:",
            example_selector=example_selector,
            example_prompt=example_prompt,
            suffix="input: {text}.\noutput:",
            input_variables=["text"],
        )
        completion_chain = few_shot_prompt | llm
        
        custom_parser = NumberParser()
        retry_parser = RetryOutputParser(parser=custom_parser, max_retries=3)

        #tqdm.pandas()
        
        main_chain = RunnableParallel(
            completion=completion_chain, prompt_value=few_shot_prompt
        ) | RunnableLambda(lambda x: retry_parser.parse_with_prompt(**x))
        
        
        test_data_sub = test_data_sub.progress_apply(process_row, axis=1)

        #-------- Evaluate and log results -----------
        file_name = f"data/enhanced_few_shot/test/classes_{n_classes}_shots_{num_shots}"
        if not os.path.exists(file_name):
            os.makedirs(file_name)
        test_data_sub.to_csv(os.path.join(file_name,"predictions.csv"), index=False)
        nan_count = test_data_sub['prediction'].isna().sum()
        test_data_sub['prediction'] = test_data_sub['prediction'].fillna(-1)
        #print(f"Number of NaN values in 'prediction': {nan_count}")
        report = classification_report(test_data_sub['label'], test_data_sub['prediction'], output_dict=True, zero_division=np.nan)
        with open(os.path.join(file_name,"results.json"), "w") as f:
            json.dump(report, f)

Processing n_classes:   0%|          | 0/1 [00:00<?, ?it/s]

Processing num_shots:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1720 [00:00<?, ?it/s]

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant<|eot_id|><|start_header_id|>user<|end_header_id|>

Your task is to classify a given text by assigning a label from the examples to it, Please ONLY respond with the label as a number. ONLY use labels you see in the examples, don't make up own labels.
Here are some examples:

input: I would like to change my pin.
output: 21

input: My bank transfer is still not showing up in my account.
output: 5

input: How do I locate my card?.
output:<|eot_id|><|start_header_id|>assistant<|end_header_id|>


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant<|eot_id|><|start_header_id|>user<|end_header_id|>

Your task is to classify a given text by assigning a label from the examples to it, Please ONLY respond with the label as a number. ONLY use labels you see in the examples, don't make up own labels.
Here are some examples:

input: My bank transfer is still not showing up in

KeyboardInterrupt: 

In [57]:
few_shot_prompt

FewShotPromptTemplate(input_variables=['text'], example_selector=SemanticSimilarityExampleSelector(vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x149480ea7520>, k=2, example_keys=None, input_keys=None, vectorstore_kwargs=None), example_prompt=PromptTemplate(input_variables=['input', 'output'], template='input: {input}\noutput: {output}'), suffix='input: {text}.\noutput:', prefix="Your task is to classify a given text by assigning a label from the examples to it, Please ONLY respond with the label as a number. ONLY use labels you see in the examples, don't make up own labels.\nHere are some examples:")

In [53]:
predictions = pd.read_csv(os.path.join(file_name,"predictions.csv"), index_col=False)
with open(os.path.join(file_name,"results.json"), "r") as f:
        report = json.load(f)

In [56]:
report

{'-1.0': {'precision': 0.0, 'recall': nan, 'f1-score': 0.0, 'support': 0.0},
 '5.0': {'precision': 0.875,
  'recall': 0.875,
  'f1-score': 0.875,
  'support': 40.0},
 '15.0': {'precision': 0.7407407407407407,
  'recall': 1.0,
  'f1-score': 0.851063829787234,
  'support': 40.0},
 '21.0': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 40.0},
 '25.0': {'precision': 0.6774193548387096,
  'recall': 0.525,
  'f1-score': 0.5915492957746479,
  'support': 40.0},
 '59.0': {'precision': 0.9090909090909091,
  'recall': 0.75,
  'f1-score': 0.821917808219178,
  'support': 40.0},
 'accuracy': 0.83,
 'macro avg': {'precision': 0.7003751674450598,
  'recall': 0.8300000000000001,
  'f1-score': 0.6899218222968434,
  'support': 200.0},
 'weighted avg': {'precision': 0.8404502009340717,
  'recall': 0.83,
  'f1-score': 0.8279061867562121,
  'support': 200.0}}

In [55]:
predictions

,text,label,prediction,inference_time,input_tokens_length,prompt_length
0,I payed with a card and was charged an extra fee,15,15.0,1.101417,123,625
1,I was charged a fee after using my card and I ...,15,15.0,1.067487,126,631
2,Is there a reason I was charged a fee for usin...,15,15.0,1.277539,135,680
3,I show another charge on my card from when I u...,15,15.0,1.066541,127,635
4,Why do I get charged additional fees on some p...,15,15.0,1.069096,126,647
...,...,...,...,...,...,...
195,Where is my money? I transfered it and it isn'...,5,5.0,1.067518,128,635
196,I need to know the time frame that is typical ...,5,5.0,1.279467,149,750
197,Why doesn't my balance reflect my transfer,5,5.0,1.069087,120,615
198,My account balance has not gone up even though...,5,5.0,1.071471,127,652


In [7]:
# TEST the example selector

# Select the most similar example to the input.
text = "Why does my credit card not work?"
selected_examples = example_selector.select_examples({"text": text})
print(f"Examples most similar to the input: {text}")
for example in selected_examples:
    print("\n")
    for k, v in example.items():
        print(f"{k}: {v}")

Examples most similar to the input: Why does my credit card not work?


text: Can you tell me why my card keeps getting declined every time I try to use it?
label: 25


text: Why do you keep declining my payment? I tried several times already with this card and it is just not working.
label: 25


In [8]:
enhanced_few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Text: {text}",
    input_variables=["text"],
)

print(enhanced_few_shot_prompt.format(text=text))

Text: Can you tell me why my card keeps getting declined every time I try to use it?
Label: 25

Text: Why do you keep declining my payment? I tried several times already with this card and it is just not working.
Label: 25

Text: Why does my credit card not work?


In [ ]:
random_seed = 42
n_classes_list = [1]
num_shots_list = [1]

for n_classes in tqdm(n_classes_list, desc="Processing n_classes"):
    for num_shots in tqdm(num_shots_list, desc="Processing num_shots", leave=False):
        sampled_classes = train_data['label'].sample(n_classes, random_state=random_seed).values
        train_data_sub = train_data[train_data['label'].isin(sampled_classes)]
        test_data_sub = test_data[test_data['label'].isin(sampled_classes)]

        #print("*"*50)
        #print(f"Number of sampled classes: {len(sampled_classes)}")
        #print(f"Number of samples in test_data_sub: {len(test_data_sub)}")


        #Achtung: muss hier drin stehen, da sampled_classes verwendet wird. Geht nicht sauberer
        class NumberParser(BaseOutputParser):
            def parse(self, text: str) -> Any:
                try:
                    # Extracting the first number from the text
                    number = int(text.strip())
                    if number not in sampled_classes:
                        raise ValueError(f"Number {number} is not in the list of valid classes.")
                    return number
                except ValueError as e:
                    raise ValueError(f"Failed to parse number: {e}")
        

        examples = get_examples(test_data_sub, num_shots)
        example_prompt = PromptTemplate(
            input_variables=["input", "output"], template="input: {input}\noutput: {output}"
        )
        few_shot_prompt = FewShotPromptTemplate(
            prefix="Your task is to classify a given text by assigning a label from the examples to it, Please ONLY respond with the label as a number. ONLY use labels you see in the examples, don't make up own labels.\nHere are some examples:",
            examples=examples,
            example_selector=example_selector,
            example_prompt=example_prompt,
            suffix="input: {text}.\noutput:",
            input_variables=["text"],
        )
        completion_chain = few_shot_prompt | llm
        
        custom_parser = NumberParser()
        retry_parser = RetryOutputParser(parser=custom_parser, max_retries=3)
        
        main_chain = RunnableParallel(
            completion=completion_chain, prompt_value=few_shot_prompt
        ) | RunnableLambda(lambda x: retry_parser.parse_with_prompt(**x))
        
        
        test_data_sub = test_data_sub.apply(process_row, axis=1)

        #-------- Evaluate and log results -----------
        file_name = f"data/enhanced_few_shot/test/classes_{n_classes}_shots_{num_shots}"
        if not os.path.exists(file_name):
            os.makedirs(file_name)
        test_data_sub.to_csv(os.path.join(file_name,"predictions.csv"))
        nan_count = test_data_sub['prediction'].isna().sum()
        test_data_sub['prediction'] = test_data_sub['prediction'].fillna(0)
        #print(f"Number of NaN values in 'prediction': {nan_count}")
        report = classification_report(test_data_sub['label'], test_data_sub['prediction'], output_dict=True, zero_division=np.nan)
        with open(os.path.join(file_name,"results.json"), "w") as f:
            json.dump(report, f)

In [13]:
response = llm(query=enhanced_few_shot_prompt.format(text=text))
response

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


86
460


"I'd be happy to help you troubleshoot the issue with your credit card!\n\nThere could be several reasons why your card is being declined. Here are a few common causes:\n\n1. **Insufficient funds**: Make sure you have enough available credit on your card to cover the transaction.\n2. **Expired card**: Check the expiration date on your card to ensure it's still valid.\n3. **Invalid card information**: Double-check that you're entering the correct card number, expiration date, and security code.\n4. **Card issuer restrictions**: Your card issuer might have placed a hold on your account or restricted certain types of transactions.\n5. **Card security features**: Some cards have security features that can cause transactions to be declined, such as a chip or a digital signature.\n6. **Merchant-specific issues**: The merchant's payment processing system might be experiencing technical difficulties or have specific requirements for card acceptance.\n\nTo resolve the issue, you can try the fol

In [9]:
def get_prompt_length(llm, str_input):
    messages = [
            {"role": "system", "content": llm._system_msg},
            {"role": "user", "content": str_input},
        ]
    prompt = llm.pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )
    input_tokens_length = len(llm.pipeline.tokenizer.encode(prompt))
    prompt_length = len(prompt)
    return input_tokens_length,prompt_length

In [10]:
input_tokens_length, prompt_length = get_prompt_length(llm, enhanced_few_shot_prompt.format(text=text))

In [11]:
input_tokens_length

86

In [12]:
prompt_length

460

In [38]:
enhanced_few_shot_prompt.format(text=text)

'Text: Why do you keep declining my payment? I tried several times already with this card and it is just not working.\nLabel: 25\n\nText: What is the reason my payment was not accepted?\nLabel: 25\n\nText: Your task is to classify a given text into one of the following classes, reply ONLY with the class label: \n\nLabel: 25\nText: Why are you declining my payment? Everything was fine.\nText: I have a card payment that was declined, but why?\n\nLabel: 21\nText: I would like to change my pin.\nText: How can I change my Tholepin ?\n\nHere is your text, please classify it into one of the above classes\n\nText: my credit card does not work'

In [30]:
class Prediction(BaseModel):
    label: int = Field(description="classification label to a given text")
    @validator("label")
    def label_is_valid(cls, field): 
        if not (0 <= field <= 76):
            raise ValueError("label must be an integer between 0 and 76")
        return field
parser = PydanticOutputParser(pydantic_object=Prediction)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
enhanced_few_shot_prompt = FewShotPromptTemplate(
    prefix="Answer the user query.\n{format_instructions}\n{text}\n",
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Text: {text}",
    input_variables=["text"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

retry_parser = RetryOutputParser.from_llm(parser=parser, llm=llm, max_retries=3)
 
completion_chain = enhanced_few_shot_prompt | llm

main_chain = RunnableParallel(
    completion=completion_chain, prompt_value=enhanced_few_shot_prompt
) | RunnableLambda(lambda x: retry_parser.parse_with_prompt(**x))


answer: Prediction = main_chain.invoke({"text": text})
input_tokens_length, prompt_length = get_prompt_length(llm, enhanced_few_shot_prompt.format(text=text))

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


242
1045


In [31]:
input_tokens_length

242

In [29]:
enhanced_few_shot_prompt.format(text=text)

'Answer the user query.\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"label": {"title": "Label", "description": "classification label to a given text", "type": "integer"}}, "required": ["label"]}\n```\nWhy does my credit card not work?\n\n\nText: Can you tell me why my card keeps getting declined every time I try to use it?\nLabel: 25\n\nText: why couldn\'t I use my card at a store?\nLabel: 25\n\nText: Why does my credit card not work?'

In [14]:
main_chain.invoke({"query": text})

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


KeyError: "Input to PromptTemplate is missing variables {'completion'}.  Expected: ['completion', 'prompt'] Received: ['prompt', 'input']"

In [ ]:
random_seed = 42
n_classes = 2
num_shots = 2
sampled_classes = train_data['label'].sample(n_classes, random_state=random_seed).values
train_data_sub = train_data[train_data['label'].isin(sampled_classes)]
test_data_sub = test_data[test_data['label'].isin(sampled_classes)]

def get_prompt_template(train_data_sub, num_shots):
    prompt_template = "Your task is to classify a given text into one of the following classes, reply ONLY with the class label: \n\n"
    for label in train_data_sub.label.unique():
        prompt_template += f"Label: {label}\n"
        for i, row in train_data_sub[train_data_sub['label'] == label].sample(num_shots, random_state=random_seed).iterrows():
            prompt_template += f"Text: {row['text']}\n"
            #remove row from the dataframe
            train_data_sub = train_data_sub.drop(i)
        prompt_template += "\n"
    prompt_template += "Here is your text, please classify it into one of the above classes\n\n"
    return prompt_template

#prompt_template = get_prompt_template(test_data_sub,num_shots)

text = get_prompt_template(test_data_sub,num_shots)
text += "Text: my credit card does not work"

### TODO:
- 

In [15]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"label": {"title": "Label", "description": "classification label to a given text", "type": "integer"}}, "required": ["label"]}\n```'

In [25]:

for i, row in tqdm.tqdm(test_data_sub.iterrows(),total=len(test_data_sub), desc="Processing data"):
    prompt = prompt_template + f"Text: {row['text']}\n"
    response = llm(prompt=prompt)
    test_data_sub.loc[i, 'response'] = response
    #print(f"Prompt: {prompt}")
    #print(f"Response: {response}")
    print("\n\n")

Processing data:   0%|          | 0/80 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
/scratch/slurm_tmpdir/job_23779533/ipykernel_169380/1091179139.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data_sub.loc[i, 'response'] = response
Processing data:   1%|▏         | 1/80 [00:16<21:17, 16.18s/it]Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Processing data:   2%|▎         | 2/80 [00:16<09:01,  6.94s/it]Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Processing data:   4%|▍         | 3/80 [00:17<05:06,  3.98s/it]Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


...

Processing data:  95%|█████████▌| 76/80 [00:51<00:01,  2.14it/s]Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Processing data:  96%|█████████▋| 77/80 [00:51<00:01,  2.14it/s]Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Processing data:  98%|█████████▊| 78/80 [00:52<00:00,  2.14it/s]Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Processing data:  99%|█████████▉| 79/80 [00:52<00:00,  2.06it/s]Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Processing data: 100%|██████████| 80/80 [00:53<00:00,  1.51it/s]

In [26]:
test_data_sub

,text,label,response
40,Why won't my card show up on the app?,13,13
41,I would like to reactivate my card.,13,13
42,Where do I link the new card?,13,13
43,"I have received my card, can you help me put i...",13,13
44,How do I link a card that I already have?,13,13
...,...,...,...
1955,The ATM at Metro bank on High St. Kensington d...,18,18
1956,The ATM won't give back my card,18,18
1957,I was at an ATM and it swallowed my card.,18,18
1958,WTF??? I tried to withdraw some money at a Met...,18,13


In [27]:
test_data_sub['response'] = test_data_sub['response'].astype(int)


/scratch/slurm_tmpdir/job_23779533/ipykernel_169380/3811748138.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data_sub['response'] = test_data_sub['response'].astype(int)


In [28]:
print(classification_report(test_data_sub['label'], test_data_sub['response']))

              precision    recall  f1-score   support

          13       0.83      1.00      0.91        40
          18       1.00      0.80      0.89        40

    accuracy                           0.90        80
   macro avg       0.92      0.90      0.90        80
weighted avg       0.92      0.90      0.90        80



In [21]:
print(classification_report(test_data_sub['label'], test_data_sub['response']))

              precision    recall  f1-score   support

          13       0.58      0.88      0.70        40
          18       0.75      0.38      0.50        40

    accuracy                           0.62        80
   macro avg       0.67      0.62      0.60        80
weighted avg       0.67      0.62      0.60        80



In [13]:
#iterate over the test data and generate the prompt
#surpress warnings
import warnings

warnings.filterwarnings("ignore")

prompt_template = get_prompt_template(test_data_sub, num_shots)

n_classes_list = [5, 10, 20, 30, 40, 50, 77]
num_shots = 5
results = {}
#iterate over the n_classes_list

# import wandb 
# wandb.init(project="llm-banking77")

from sklearn.metrics import classification_report

for n_classes in n_classes_list:
    sampled_classes = train_data['label'].sample(n_classes, random_state=random_seed).values
    train_data_sub = train_data[train_data['label'].isin(sampled_classes)]
    test_data_sub = test_data[test_data['label'].isin(sampled_classes)]
    prompt_template = get_prompt_template(test_data_sub, num_shots)

    for i, row in tqdm.tqdm(test_data_sub.iterrows(),total=len(test_data_sub), desc="Processing data"):
        prompt = prompt_template + f"Text: {row['text']}\n"
        response = llm(prompt=prompt)
        test_data_sub.loc[i, 'response'] = response
    test_data_sub['response'] = test_data_sub['response'].label.astype(int)
    report = classification_report(test_data_sub['label'], test_data_sub['response'], output_dict=True)
    # #print(f"Classification Report: {classification_report}")
    # #save results to dictionary
    # results[n_classes] = {}
    results[n_classes] = {
        'classification_report': report,
        'prompt_template': prompt_template,
        'context_length': len(prompt_template)
    }
    
 

Processing data:   0%|          | 0/200 [00:00<?, ?it/s]


TypeError: __call__() missing 1 required keyword-only argument: 'query'

In [ ]:
import json
with open("../data/results.json", "w") as f:
    json.dump(results, f)